In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from PIL import Image, ImageStat

In [2]:
wb = load_workbook('Послойное описание с фотографиями.xlsx')
ws = wb[wb.sheetnames[0]]
df = pd.DataFrame(ws.values).loc[4:56,14:17]
df.head(53)

,14,15,16,17
4,песчаник,не карбонатный,не разрушен,нефтенасыщенные
5,алевролит,не карбонатный,не разрушен,не опред.
6,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные
7,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные
8,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные
9,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные
10,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные
11,алевролит,не карбонатный,не разрушен,не опред.
12,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные
13,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные


In [3]:
oil_imgs=[]
path = 'pics/'
i = 0
for row in df.itertuples():
    i += 1
    title=str(i+4)+' УФ\n'+row[2]+'\n'+row[4]
    #if 'не карбонатный' in row[2]:
    #    continue
    elems = []
    for elem in os.listdir(path+str(i)):
        elems.append(path+str(i)+'/'+elem)
    src = elems[:int(len(elems)/2)]
    uv = elems[int(len(elems)/2):]
    src = np.concatenate([cv2.imread(img,1) for img in src], axis=0)
    uv = np.concatenate([cv2.imread(img,1) for img in uv], axis=0)
    img_dn = cv2.fastNlMeansDenoisingColored(uv, h=10) 
    #img_dn = cv2.resize(img_dn,None,fx=1,fy=2,interpolation=cv2.INTER_CUBIC)
    img_rgb = cv2.cvtColor(img_dn, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img_dn, cv2.COLOR_BGR2HSV)
    #img_h = np.array([[[x[0],255,255] for x in y] for y in img_hsv])
    pil_img = Image.fromarray(img_rgb)
    bright = sum(ImageStat.Stat(pil_img).mean) // 3
    img_gray = cv2.cvtColor(img_dn,cv2.COLOR_BGR2GRAY)
    dark = (85*(bright/85),100*(bright/85),100*(bright/85))
    light = (255,255,255)
    mask_yel = cv2.inRange(img_rgb, dark, light)
    img_yel = cv2.bitwise_and(img_gray, img_gray, mask=mask_yel)
    img_yel = cv2.GaussianBlur(img_yel, (5,5), 0)
    oil_imgs.append(img_yel)
    thresh = (0,0,10)
    mask_thresh = cv2.inRange(img_hsv, thresh, (180,255,255))
    img_hsv = cv2.bitwise_and(img_hsv, img_hsv, mask=mask_thresh)
    pil_img2 = Image.fromarray(img_hsv)
    mean_stat = ImageStat.Stat(pil_img2).mean
    mean_hue = mean_stat[0]
    mean_sat = mean_stat[1]
    mean_val = mean_stat[2]
    dark = (0,10,10)
    light = (30,255,255)
    mask_red = cv2.inRange(img_hsv, dark, light)
    img_red = cv2.bitwise_and(img_gray, img_gray, mask=mask_red)
    img_red = cv2.GaussianBlur(img_red, (5,5), 0)
    '''
    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(1, 5, figsize=(15,100), sharex=True, sharey=True)

    ax1.imshow(img_rgb)
    ax1.set_title(title+'\nRGB Denoised\nMean Hue: '+str(mean_hue)+
                  '\nMean Saturation: '+str(mean_sat)+'\nMean Value: '+str(mean_val))
        
    ax2.imshow(img_gray)
    ax2.set_title('Gray')
    
    ax4.imshow(img_yel, cmap='gray')
    ax4.set_title('Oil mask')
    
    ax3.imshow(img_hsv)
    ax3.set_title('HSV (Value=10+)')
    
    ax5.imshow(img_red, cmap='gray')
    ax5.set_title('Carbon mask')
    plt.show()
    '''

In [4]:
def surf(img):
    surf = cv2.xfeatures2d.SURF_create(2000)
    kp, des = surf.detectAndCompute(img,None)
    return cv2.drawKeypoints(img,kp,None,(255,0,0),4)


In [5]:
opil_imgs = [Image.fromarray(img) for img in oil_imgs]
oil_means = [sum(ImageStat.Stat(img).mean) for img in opil_imgs]
df['Oil Means'] = oil_means

In [6]:
best_cor = 0
par = 5
step = 0.01
df['Oil Preds'] = [0]*52
for p in np.arange(5.,50.,step):
    oil_preds = [1 if mean > p else 0 for mean in oil_means]
    cor = 0
    i = 0
    for row in df.itertuples():
        if row[4] == 'не опред.':
            if oil_preds[i] == 0:
                cor += 1
        else:
            if oil_preds[i] == 1:
                cor += 1
        i += 1
    if cor > best_cor:
        par = p - step
        best_cor = cor
        df['Oil Preds'] = oil_preds
    
print('param: ', par)
print('correct: ', best_cor)
print('accuracy: ', best_cor/52.)

param:  7.899999999999938
correct:  43
accuracy:  0.8269230769230769


In [7]:
def area_part(arr):
    pxls = sum([sum([0 if pxl == 0 else 1 for pxl in row])for row in arr])
    return pxls/(len(arr)*len(arr[0]))
oil_areas = [area_part(img) for img in oil_imgs]
df['Oil Areas'] = oil_areas

In [8]:
best_cor = 0
par = 0
step = 0.005
df['Oil Preds #2'] = [0]*52
for p in np.arange(0.,1.,step):
    oil_area_preds = [1 if area > p else 0 for area in oil_areas]
    cor = 0
    i = 0
    for row in df.itertuples():
        if row[4] == 'не опред.':
            if oil_area_preds[i] == 0:
                cor += 1
        else:
            if oil_area_preds[i] == 1:
                cor += 1
        i += 1
    if cor > best_cor:
        par = p - step
        best_cor = cor
        df['Oil Preds #2'] = oil_area_preds
print('param: ', par)
print('correct: ', best_cor)
print('accuracy: ', best_cor/52.)

param:  0.265
correct:  44
accuracy:  0.8461538461538461


In [9]:
df.head(53)

,14,15,16,17,Oil Means,Oil Preds,Oil Areas,Oil Preds #2
4,песчаник,не карбонатный,не разрушен,нефтенасыщенные,31.534172,1,0.562649,1
5,алевролит,не карбонатный,не разрушен,не опред.,6.488578,0,0.296743,1
6,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,43.689167,1,0.443583,1
7,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные,10.116250,1,0.607500,1
8,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,42.058550,1,0.438779,1
9,"пересл. песчаника, алевролита и глин",не карбонатный,не разрушен,пятнисто нефтенасыщенные,25.139610,1,0.631299,1
10,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,4.570323,0,0.197742,0
11,алевролит,не карбонатный,не разрушен,не опред.,34.654000,1,0.530160,1
12,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,21.743548,1,0.375323,1
13,песчаник,не карбонатный,не разрушен,слабо нефтенасыщенные,33.183692,1,0.421231,1


In [10]:
X = [[oil_means[i], oil_areas[i]] for i in range(len(oil_means))]
y = [0. if oil=='не опред.' else 1. for oil in df[17]]

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=10, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_test)
c = 0
for i in range(len(preds)):
    if preds[i] > 0.5:
        if y_test[i] == 1:
            c += 1
    else:
        if y_test[i] == 0:
            c += 1

print(c/len(preds))

0.8571428571428571


In [22]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

xgb_model = XGBRegressor(n_estimators=10, learning_rate=0.5)
scores = -1 * cross_val_score(xgb_model, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print('MAE: ', scores.mean())
xgb_model.fit(X_train, y_train)
preds = xgb_model.predict(X_test)
c = 0
for i in range(len(preds)):
    if preds[i] > 0.5:
        if y_test[i] == 1:
            c += 1
    else:
        if y_test[i] == 0:
            c += 1
            
print(c/len(preds))

[02:54:17] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:17] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:17] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:17] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:17] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE:  0.3415192663669586
[02:54:17] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.7142857142857143
